In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
import openai
# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
    
print("Project root added to sys.path:", project_root)



Project root added to sys.path: /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2


In [27]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)


Project root added to sys.path: /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2


In [28]:
from src.schema import LedgerRow

row = LedgerRow(
    doc_id="ledger1",
    page_id=1,
    row_id=0,
    description="Payment for hay",
    transaction_type="Debit",
    pounds=3,
    shillings=4,
    pence=11,
    pence_fraction="1/4",
    model_conf_description=0.9,
    model_conf_transaction_type=0.95,
    model_conf_pounds=0.88,
    model_conf_shillings=0.85,
    model_conf_pence=0.92,
    model_conf_pence_fraction=0.75,
    rule_based_confidence=0.80,
    row_confidence=0.83,
)

row


{'doc_id': 'ledger1',
 'page_id': 1,
 'row_id': 0,
 'description': 'Payment for hay',
 'transaction_type': 'Debit',
 'pounds': 3,
 'shillings': 4,
 'pence': 11,
 'pence_fraction': '1/4',
 'model_conf_description': 0.9,
 'model_conf_transaction_type': 0.95,
 'model_conf_pounds': 0.88,
 'model_conf_shillings': 0.85,
 'model_conf_pence': 0.92,
 'model_conf_pence_fraction': 0.75,
 'rule_based_confidence': 0.8,
 'row_confidence': 0.83}

In [29]:
from src.schema import PageMetadata

page_meta: PageMetadata = {
    "doc_id": "ledger1",
    "page_id": 1,
    "page_type": "Full_Balance_Sheet",
    "financial_structure_overview": "Page summarises income and expenditure with separate debit/credit sections."
}

page_meta


{'doc_id': 'ledger1',
 'page_id': 1,
 'page_type': 'Full_Balance_Sheet',
 'financial_structure_overview': 'Page summarises income and expenditure with separate debit/credit sections.'}

In [30]:
from src.classifier import build_classifier_prompt

sample_prompt = build_classifier_prompt(
    doc_id="ledger1",
    page_id=1,
    page_text="RENT RECEIVED FROM JOHN SMITH\n£3 4s 11d\nTOTAL INCOME FOR THE YEAR"
)

print(sample_prompt[:500])  # print only the first 500 characters for sanity


You are a financial document analysis assistant. You analyze historical ledger pages
and classify the role of each page within the ledger.

You will be given:
- The document ID (doc_id)
- The page index (page_id)
- The raw text content extracted from that page

Your task is to:
1. Classify the type of the page.
2. Provide a short, human-readable overview of the financial structure on this page.

There are THREE possible page types:

- "Full_Balance_Sheet":
  The page primarily summarises income 


In [31]:
from src.classifier import classify_page

sample_text = """RENT RECEIVED FROM JOHN SMITH
£3 4s 11d
TOTAL INCOME FOR THE YEAR"""

meta = classify_page(
    doc_id="ledger1",
    page_id=1,
    page_text=sample_text,
)

meta


{'doc_id': 'ledger1',
 'page_id': 1,
 'page_type': 'Full_Balance_Sheet',
 'financial_structure_overview': "Fallback: classified as balance/summary page based on presence of 'total' or 'balance'."}

In [32]:
from src.schema import PageMetadata
from src.extractor import build_extraction_prompt

page_meta: PageMetadata = {
    "doc_id": "ledger1",
    "page_id": 2,
    "page_type": "Sectional_List",
    "financial_structure_overview": "List of individual rent payments by tenant."
}

sample_text = """JOHN SMITH - RENT
£3 4s 11d
MARY JONES - RENT
£2 10s 0d"""

prompt = build_extraction_prompt(page_meta, sample_text)

print(prompt[:700])  # just to avoid printing a huge wall

You are a financial document extraction assistant. You analyze historical ledgers
and extract individual financial entries in a clean, machine-readable format.

You will be given:
- The document ID (doc_id)
- The page index (page_id)
- The page type (page_type), as classified by a previous step
- A short description of the page's financial structure (financial_structure_overview)
- The raw text content of that page (PAGE_TEXT)

Your task is to extract a list of individual ledger entries from this page. Each
entry should correspond to a single transaction or line item whenever possible.

IMPORTANT CONCEPTS:

1. VERTICAL STACKING OF ENTRIES
   Historical ledgers often show DEBIT and CREDIT as 


In [33]:
from src.schema import PageMetadata
from src.extractor import extract_page

page_meta: PageMetadata = {
    "doc_id": "ledger1",
    "page_id": 3,
    "page_type": "Sectional_List",
    "financial_structure_overview": "List of rent payments."
}

sample_text = """JOHN SMITH - RENT £3 4s 11d
MARY JONES - RENT £2 10s 0d"""

rows = extract_page(page_meta, sample_text)

rows


[{'doc_id': 'ledger1',
  'page_id': 3,
  'row_id': 0,
  'description': 'Dummy individual transaction inferred from list-like page text.',
  'transaction_type': 'Unknown',
  'pounds': None,
  'shillings': None,
  'pence': None,
  'pence_fraction': None,
  'model_conf_description': 0.3,
  'model_conf_transaction_type': 0.2,
  'model_conf_pounds': 0.0,
  'model_conf_shillings': 0.0,
  'model_conf_pence': 0.0,
  'model_conf_pence_fraction': 0.0,
  'rule_based_confidence': 0.7,
  'row_confidence': 0.32999999999999996}]

In [34]:
from src.pipeline import process_single_page

sample_text = """RENT RECEIVED FROM JOHN SMITH
£3 4s 11d
TOTAL INCOME FOR THE YEAR"""

page_meta, rows = process_single_page(
    doc_id="ledger1",
    page_id=1,
    page_text=sample_text,
)

page_meta, rows


({'doc_id': 'ledger1',
  'page_id': 1,
  'page_type': 'Full_Balance_Sheet',
  'financial_structure_overview': "Fallback: classified as balance/summary page based on presence of 'total' or 'balance'."},
 [{'doc_id': 'ledger1',
   'page_id': 1,
   'row_id': 0,
   'description': 'Dummy balance entry inferred from summary-like page text.',
   'transaction_type': 'Unknown',
   'pounds': None,
   'shillings': None,
   'pence': None,
   'pence_fraction': None,
   'model_conf_description': 0.3,
   'model_conf_transaction_type': 0.2,
   'model_conf_pounds': 0.0,
   'model_conf_shillings': 0.0,
   'model_conf_pence': 0.0,
   'model_conf_pence_fraction': 0.0,
   'rule_based_confidence': 0.7,
   'row_confidence': 0.32999999999999996}])

In [35]:
from src.utils import load_env, get_openai_api_key

# load .env
load_env()

# try to read key
try:
    key = get_openai_api_key()
    print("API key loaded (first 5 chars):", key[:5], "...")
except RuntimeError as e:
    print("Error:", e)


API key loaded (first 5 chars): sk-.. ...


In [36]:
from src.schema import LedgerRow
from src.scorer import compute_rule_based_confidence, compute_row_confidence

sample_row: LedgerRow = {
    "doc_id": "ledger1",
    "page_id": 1,
    "row_id": 0,
    "description": "Rent received from John Smith",
    "transaction_type": "Credit",
    "pounds": 3,
    "shillings": 4,
    "pence": 11,
    "pence_fraction": "1/4",
    "model_conf_description": 0.9,
    "model_conf_transaction_type": 0.95,
    "model_conf_pounds": 0.88,
    "model_conf_shillings": 0.85,
    "model_conf_pence": 0.92,
    "model_conf_pence_fraction": 0.75,
    "rule_based_confidence": 0.0,  # will be recomputed
    "row_confidence": 0.0,         # will be recomputed
}

rule_conf = compute_rule_based_confidence(sample_row, typical_max_pounds=20)
row_conf = compute_row_confidence(sample_row, rule_weight=0.4, typical_max_pounds=20)

rule_conf, row_conf


(1.0, 0.925)

In [2]:
from src.pipeline import process_single_page

sample_text = """RENT RECEIVED FROM JOHN SMITH
£3 4s 11d
TOTAL INCOME FOR THE YEAR"""

page_meta, rows = process_single_page(
    doc_id="ledger1",
    page_id=1,
    page_text=sample_text,
)

page_meta, rows


({'doc_id': 'ledger1',
  'page_id': 1,
  'page_type': 'Full_Balance_Sheet',
  'financial_structure_overview': 'Summary of total income received from rent, specifically from John Smith, with a total income figure for the year.'},
 [{'doc_id': 'ledger1',
   'page_id': 1,
   'row_id': 0,
   'description': 'Dummy balance entry inferred from summary-like page text.',
   'transaction_type': 'Unknown',
   'pounds': None,
   'shillings': None,
   'pence': None,
   'pence_fraction': None,
   'model_conf_description': 0.3,
   'model_conf_transaction_type': 0.2,
   'model_conf_pounds': 0.0,
   'model_conf_shillings': 0.0,
   'model_conf_pence': 0.0,
   'model_conf_pence_fraction': 0.0,
   'rule_based_confidence': 0.7,
   'row_confidence': 0.32999999999999996}])

In [38]:
import sys
print(sys.executable)


/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/venv/bin/python


In [3]:
page_meta, rows = process_single_page(
    doc_id="ledger1",
    page_id=1,
    page_text=sample_text,
)

page_meta, rows


({'doc_id': 'ledger1',
  'page_id': 1,
  'page_type': 'Full_Balance_Sheet',
  'financial_structure_overview': 'Summary of total income received from rent, specifically from John Smith, with a total income figure for the year.'},
 [{'doc_id': 'ledger1',
   'page_id': 1,
   'row_id': 0,
   'description': 'RENT RECEIVED FROM JOHN SMITH',
   'transaction_type': 'Credit',
   'pounds': 3,
   'shillings': 4,
   'pence': 11,
   'pence_fraction': None,
   'model_conf_description': 1.0,
   'model_conf_transaction_type': 1.0,
   'model_conf_pounds': 1.0,
   'model_conf_shillings': 1.0,
   'model_conf_pence': 1.0,
   'model_conf_pence_fraction': 1.0,
   'rule_based_confidence': 1.0,
   'row_confidence': 1.0}])

In [4]:
from src.pipeline import process_document

pages = {
    1: """RENT RECEIVED FROM JOHN SMITH
£3 4s 11d
TOTAL INCOME FOR THE YEAR""",
    2: """RENT RECEIVED FROM MARY JONES
£2 10s 0d
ADDITIONAL NOTES""",
}

doc_meta, doc_rows = process_document(
    doc_id="ledger_demo",
    pages=pages,
)

doc_meta, doc_rows


([{'doc_id': 'ledger_demo',
   'page_id': 1,
   'page_type': 'Full_Balance_Sheet',
   'financial_structure_overview': 'Summary of total income received from rent, specifically from John Smith, with a total income figure for the year.'},
  {'doc_id': 'ledger_demo',
   'page_id': 2,
   'page_type': 'Sectional_List',
   'financial_structure_overview': 'List of individual rent payment received from Mary Jones.'}],
 [{'doc_id': 'ledger_demo',
   'page_id': 1,
   'row_id': 0,
   'description': 'RENT RECEIVED FROM JOHN SMITH',
   'transaction_type': 'Credit',
   'pounds': 3,
   'shillings': 4,
   'pence': 11,
   'pence_fraction': None,
   'model_conf_description': 1.0,
   'model_conf_transaction_type': 1.0,
   'model_conf_pounds': 1.0,
   'model_conf_shillings': 1.0,
   'model_conf_pence': 1.0,
   'model_conf_pence_fraction': 1.0,
   'rule_based_confidence': 1.0,
   'row_confidence': 1.0},
  {'doc_id': 'ledger_demo',
   'page_id': 2,
   'row_id': 0,
   'description': 'RENT RECEIVED FROM MARY 

In [6]:
from src.loader import load_pdf_as_pages

pages = load_pdf_as_pages("1704")
len(pages), list(pages.keys())[:5]


(7, [1, 2, 3, 4, 5])

In [7]:
from src.loader import load_pdf_as_pages
from src.pipeline import process_document

# 1) Load pages from the real PDF
pages_1704 = load_pdf_as_pages("1704")
print("Number of pages:", len(pages_1704))

# 2) Run the full LLM pipeline on this document
doc_meta_1704, doc_rows_1704 = process_document(
    doc_id="1704",
    pages=pages_1704,
)

len(doc_meta_1704), len(doc_rows_1704)


Number of pages: 7


(7, 0)

In [8]:
from src.loader import export_pdf_pages_as_images

image_paths_1704 = export_pdf_pages_as_images("1704")
image_paths_1704


{1: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_1.png',
 2: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_2.png',
 3: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_3.png',
 4: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_4.png',
 5: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_5.png',
 6: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_6.png',
 7: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_7.png'}

In [10]:
from src.loader import export_pdf_pages_as_images
from src.ocr import ocr_page_with_gpt

# Ensure images exist (already done, but safe to rerun)
image_paths_1704 = export_pdf_pages_as_images("1704")
image_paths_1704



{1: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_1.png',
 2: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_2.png',
 3: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_3.png',
 4: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_4.png',
 5: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_5.png',
 6: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_6.png',
 7: '/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_7.png'}

In [12]:
page1_img = image_paths_1704[1]
print("Using image:", page1_img)

page1_text = ocr_page_with_gpt(page1_img)
print(page1_text[:1000])  # show first 1000 chars

Using image: /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_1.png
```
Comptey que: Paynter Rectory a secundo die 
Novembris 1703, ad eundem diem 1704

Jmprims redidit compt de remanentib. in lista
11  9  46
Glen de arerraig's
Guineas                     66  0  0
Longwitnam                 76  13  4
minchinett                 20  0  0
Southernwington            30  0  0
Merton
Chacehill
Thrup                      03  8  0
Tye quittrents            02  15  7
Ralph marshall capons
Calcott                    03  8  11
Bampton                    04  0  0
Mountrouse
Heughes                    01  12  0
Greenway
Walker
minne                     00  10  0
Leaden-hall
Molineux
Tintinhull                { Napper
                          Hopkins
Clifton Ferry             02  5  59
Bensington                01  18  7
Brew house                03  19  8
Trethewin                 01  15  6
Merton Vicar.             06  9  7
             

In [13]:
from src.ocr import ocr_pdf_to_pages
from src.pipeline import process_document

# 1) OCR all pages of 1704.pdf
pages_1704_ocr = ocr_pdf_to_pages("1704")  # This will take a bit + print progress
print("Pages OCR'd:", len(pages_1704_ocr))

# 2) Run the existing LLM pipeline on that OCR text
doc_meta_1704, doc_rows_1704 = process_document(
    doc_id="1704",
    pages=pages_1704_ocr,
)

len(doc_meta_1704), len(doc_rows_1704)


[OCR] 1704 page 1 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_1.png
[OCR] 1704 page 2 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_2.png
[OCR] 1704 page 3 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_3.png
[OCR] 1704 page 4 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_4.png
[OCR] 1704 page 5 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_5.png
[OCR] 1704 page 6 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_6.png
[OCR] 1704 page 7 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_7.png
Pages OCR'd: 7


(7, 115)

In [14]:
doc_meta_1704[:3]   # first 3 pages' metadata

doc_rows_1704[:10]  # first 10 extracted rows


[{'doc_id': '1704',
  'page_id': 1,
  'row_id': 0,
  'description': 'Guineas',
  'transaction_type': 'Credit',
  'pounds': 66,
  'shillings': 0,
  'pence': 0,
  'pence_fraction': None,
  'model_conf_description': 1.0,
  'model_conf_transaction_type': 1.0,
  'model_conf_pounds': 1.0,
  'model_conf_shillings': 1.0,
  'model_conf_pence': 1.0,
  'model_conf_pence_fraction': 1.0,
  'rule_based_confidence': 1.0,
  'row_confidence': 1.0},
 {'doc_id': '1704',
  'page_id': 1,
  'row_id': 1,
  'description': 'Longwitnam',
  'transaction_type': 'Credit',
  'pounds': 76,
  'shillings': 13,
  'pence': 4,
  'pence_fraction': None,
  'model_conf_description': 1.0,
  'model_conf_transaction_type': 1.0,
  'model_conf_pounds': 1.0,
  'model_conf_shillings': 1.0,
  'model_conf_pence': 1.0,
  'model_conf_pence_fraction': 1.0,
  'rule_based_confidence': 1.0,
  'row_confidence': 1.0},
 {'doc_id': '1704',
  'page_id': 1,
  'row_id': 2,
  'description': 'minchinett',
  'transaction_type': 'Credit',
  'pounds'

In [15]:
import pandas as pd
import os

# Assume doc_rows_1704 is already in memory from process_document
df_1704 = pd.DataFrame(doc_rows_1704)

# Show a quick preview
df_1704.head()


,doc_id,page_id,row_id,description,transaction_type,pounds,shillings,pence,pence_fraction,model_conf_description,model_conf_transaction_type,model_conf_pounds,model_conf_shillings,model_conf_pence,model_conf_pence_fraction,rule_based_confidence,row_confidence
0,1704,1,0,Guineas,Credit,66.0,0.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1704,1,1,Longwitnam,Credit,76.0,13.0,4.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1704,1,2,minchinett,Credit,20.0,0.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1704,1,3,Southernwington,Credit,30.0,0.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1704,1,4,Merton,Credit,3.0,8.0,0.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
from src.batch import process_all_pdfs_to_excel

out_path = process_all_pdfs_to_excel(output_excel_name="all_years_rows.xlsx")
out_path



=== Processing 1704 ===
[OCR] 1704 page 1 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_1.png
[OCR] 1704 page 2 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_2.png
[OCR] 1704 page 3 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_3.png
[OCR] 1704 page 4 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_4.png
[OCR] 1704 page 5 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_5.png
[OCR] 1704 page 6 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_6.png
[OCR] 1704 page 7 -> /Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/interim/1704/page_7.png
Finished 1704: pages=7,

'/Users/hamidostadi/Documents/0. Research/H-AI KHu lab/Week2 Task/ledger_extraction_v2/data/processed/all_years_rows.xlsx'

In [17]:
import pandas as pd
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
df = pd.read_excel(
    os.path.join(project_root, "data", "processed", "all_years_rows.xlsx"),
    sheet_name="rows"
)

df.shape


(4815, 17)

In [18]:
df["row_confidence"].describe()


count    4815.000000
mean        0.849046
std         0.209418
min         0.270000
25%         0.800000
50%         0.950000
75%         1.000000
max         1.000000
Name: row_confidence, dtype: float64

In [19]:
df.groupby("doc_id")["row_confidence"].mean().sort_values()


doc_id
1713         0.577342
1719         0.616947
1753         0.667034
1704         0.677679
1814         0.685000
1807         0.691886
1709         0.706457
1865         0.724524
1795-1796    0.741391
1744         0.801185
1731         0.804526
1759-1760    0.816898
1774-1775    0.847618
1860         0.848065
1869         0.858129
1801         0.861172
1855         0.866295
1889         0.870690
1881         0.878423
1900         0.884058
1725         0.889878
1873         0.890076
1838         0.896125
1844         0.896973
1895         0.901613
1783-1784    0.912754
1850         0.949215
1825         0.955929
1831         0.970415
1767         0.981635
1788-1789    0.986816
1738         0.989737
1819         0.995678
Name: row_confidence, dtype: float64

In [20]:
import pandas as pd
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
df = pd.read_excel(
    os.path.join(project_root, "data", "processed", "all_years_rows.xlsx"),
    sheet_name="rows"
)

len(df), df["doc_id"].nunique()


(4815, 33)

In [21]:
df["row_confidence"].describe()


count    4815.000000
mean        0.849046
std         0.209418
min         0.270000
25%         0.800000
50%         0.950000
75%         1.000000
max         1.000000
Name: row_confidence, dtype: float64

In [22]:
df.sort_values("row_confidence", ascending=False).head(1)


,doc_id,page_id,row_id,description,transaction_type,pounds,shillings,pence,pence_fraction,model_conf_description,model_conf_transaction_type,model_conf_pounds,model_conf_shillings,model_conf_pence,model_conf_pence_fraction,rule_based_confidence,row_confidence
1764,1788-1789,1,21,Wild Arish pro annis,Credit,5.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [23]:
df.sort_values("row_confidence").head(1)


,doc_id,page_id,row_id,description,transaction_type,pounds,shillings,pence,pence_fraction,model_conf_description,model_conf_transaction_type,model_conf_pounds,model_conf_shillings,model_conf_pence,model_conf_pence_fraction,rule_based_confidence,row_confidence
3889,1865,2,21,10,Unknown,NaN,NaN,NaN,NaN,0.2,0.5,0.0,0.0,0.0,0.0,0.5,0.27
